# **SVM Classifier**
*   Implementation of a classic SVM classifier supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a classic SVM network classifier supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.

In [1]:
import time
import tensorflow as tf
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
# Set a global TF random seed
tf.random.set_seed(12)

In [2]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 5

In [10]:
def build_and_train(X_train, y_train, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Dense(3, activation='linear', dtype=dtype)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=100)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [24]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    model = models.Sequential([
        layers.Dense(3, activation='linear')
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [18]:
# Load dataset and split into train and test sets
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4715 - accuracy: 0.2833
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4560 - accuracy: 0.2833
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4414 - accuracy: 0.2833
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4260 - accuracy: 0.2750
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4121 - accuracy: 0.2750
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3973 - accuracy: 0.2667
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3829 - accuracy: 0.2667
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3696 - accuracy: 0.2667
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3548 - accuracy: 0.2667
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3409 - accuracy: 0.2667
Epoch 11/100
4/4 [===========

In [41]:
# Train and evaluate with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, precision='double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/100
4/4 [==============================] - 1s 5ms/step - loss: 2.2034 - accuracy: 0.2500
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1856 - accuracy: 0.2583
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 2.1672 - accuracy: 0.2583
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1491 - accuracy: 0.2667
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1311 - accuracy: 0.2667
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 2.1135 - accuracy: 0.2667
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0954 - accuracy: 0.2667
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0776 - accuracy: 0.2667
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0605 - accuracy: 0.2667
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0426 - accuracy: 0.2667
Epoch 11/100
4/4 [===========

In [43]:
# Train and evaluate with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, precision='single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/100
4/4 [==============================] - 1s 6ms/step - loss: 0.9369 - accuracy: 0.6417
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 0.9279 - accuracy: 0.6583
Epoch 3/100
4/4 [==============================] - 0s 6ms/step - loss: 0.9183 - accuracy: 0.6667
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 0.9096 - accuracy: 0.6583
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 0.9003 - accuracy: 0.6583
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8915 - accuracy: 0.6833
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8830 - accuracy: 0.6833
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8746 - accuracy: 0.6833
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8662 - accuracy: 0.6917
Epoch 10/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8574 - accuracy: 0.6833
Epoch 11/100
4/4 [===========

In [31]:
# Train and evaluate with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, precision='half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/100
4/4 [==============================] - 0s 5ms/step - loss: 1.0707 - accuracy: 0.4500
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0886 - accuracy: 0.4750
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0787 - accuracy: 0.4750
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0698 - accuracy: 0.4750
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0608 - accuracy: 0.4750
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0516 - accuracy: 0.4750
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 1.0426 - accuracy: 0.4917
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 1.0337 - accuracy: 0.5167
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0255 - accuracy: 0.5250
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0169 - accuracy: 0.5250
Epoch 11/100
4/4 [===========

In [45]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 0.5614 - accuracy: 0.7250
Epoch 2/5
4/4 [==============================] - 0s 5ms/step - loss: 0.5581 - accuracy: 0.7417
Epoch 3/5
4/4 [==============================] - 0s 5ms/step - loss: 0.5547 - accuracy: 0.7500
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 0.5511 - accuracy: 0.7500
Epoch 5/5
4/4 [==============================] - 0s 4ms/step - loss: 0.5479 - accuracy: 0.7583
Epoch 1/5
4/4 [==============================] - 1s 5ms/step - loss: 1.5497 - accuracy: 0.2583
Epoch 2/5
4/4 [==============================] - 0s 4ms/step - loss: 1.5340 - accuracy: 0.2583
Epoch 3/5
4/4 [==============================] - 0s 4ms/step - loss: 1.5188 - accuracy: 0.2583
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 1.5045 - accuracy: 0.2583
Epoch 5/5
4/4 [==============================] - 0s 5ms/step - loss: 1.4893 - accuracy: 0.2583
Epoch 1/5
4/4 [==============================] - 1

In [46]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 3.788557291030884 seconds
Average training time in single precision: 3.494900035858154 seconds
Average training time in half precision: 3.3682705879211428 seconds
Average training time in mixed half precision: 0.8529082775115967 seconds
-------------
Accuracy with double precision: 0.8666666746139526
Accuracy with single precision: 0.7666666507720947
Accuracy with half precision: 0.7666666507720947
Accuracy with mixed half precision: 0.6333333253860474
